In [22]:
import os
import shutil

images_dir = '/home/quangsang/test/cropped_cam10/images'
labels_dir = '/home/quangsang/test/cropped_cam10/labels'
output_dir = '/home/quangsang/test/cropped_cam10/cropped_images_by_class'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for image_name in os.listdir(images_dir):
    if image_name.lower().endswith(('.jpg', '.jpeg', '.png')):

        parts = image_name.split('_')
        if len(parts) < 2:
            print(f"Filename {image_name} does not conform to expected format. Skipping.")
            continue
        class_part = parts[-1]
        class_id = os.path.splitext(class_part)[0].replace('class_', '')

        class_name = f'class_{class_id}'

        class_dir = os.path.join(output_dir, class_name)
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)
            
            images_class_dir = os.path.join(class_dir, 'images')
            labels_class_dir = os.path.join(class_dir, 'labels')
            os.makedirs(images_class_dir)
            os.makedirs(labels_class_dir)
        else:
            images_class_dir = os.path.join(class_dir, 'images')
            labels_class_dir = os.path.join(class_dir, 'labels')

        image_output_path = os.path.join(images_class_dir, image_name)
        label_name = os.path.splitext(image_name)[0] + '.txt'
        label_output_path = os.path.join(labels_class_dir, label_name)

        source_image_path = os.path.join(images_dir, image_name)
        source_label_path = os.path.join(labels_dir, label_name)

        if not os.path.isfile(source_label_path):
            print(f"Label file {label_name} does not exist for image {image_name}. Skipping.")
            continue

        shutil.copy(source_image_path, image_output_path)
        shutil.copy(source_label_path, label_output_path)

print("Done organizing images and labels by class with separate folders for images and labels.")

Done organizing images and labels by class with separate folders for images and labels.


In [64]:
import os
import shutil

import numpy as np
import torch
from PIL import Image
from sklearn.cluster import KMeans
from tqdm import tqdm
from transformers import pipeline

# Đặt thiết bị là CUDA 1 nếu có sẵn
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# Khởi tạo pipeline để trích xuất đặc trưng từ hình ảnh
pipe = pipeline(
    task="image-feature-extraction",
    model="google/vit-base-patch16-224-in21k",
    device=DEVICE.index,  # Chỉ định thiết bị cụ thể
    pool=True,
    use_fast=True,
)

# Tạo thư mục để lưu các hình ảnh đại diện
# images_save_dir = "/home/quangsang/test/cropped_cam1/represent_images_by_class/class_3/images"
# os.makedirs(images_save_dir, exist_ok=True)
# labels_save_dir = "/home/quangsang/test/cropped_cam1/represent_images_by_class/class_3/labels"
# os.makedirs(labels_save_dir, exist_ok=True)
integration_save_dir = "/home/quangsang/test/cropped_cam10/represent_images_by_class/class_3"
os.makedirs(integration_save_dir, exist_ok=True)

# Đường dẫn tới thư mục chứa các hình ảnh đầu vào
images_root = "/home/quangsang/test/cropped_cam10/cropped_images_by_class/class_3/images"
labels_root = "/home/quangsang/test/cropped_cam10/cropped_images_by_class/class_3/labels"

# Trích xuất đặc trưng từ tất cả các hình ảnh
featurelist = []
filelist = os.listdir(images_root)
for image_name in tqdm(filelist, desc="Extracting features"):
    image_path = os.path.join(images_root, image_name)
    image = Image.open(image_path)
    outputs = pipe([image])
    feature = outputs[0][0]
    featurelist.append(feature)

# Chuyển đổi danh sách đặc trưng thành mảng numpy
features = np.array(featurelist)

# Phân cụm các đặc trưng
number_clusters = 200  # Số lượng cụm bạn muốn
kmeans = KMeans(n_clusters=number_clusters, random_state=0).fit(features)

# Tìm hình ảnh gần nhất với mỗi tâm cụm
centroid_indices = []
for centroid in kmeans.cluster_centers_:
    distances = np.linalg.norm(features - centroid, axis=1)
    closest_index = np.argmin(distances)
    centroid_indices.append(closest_index)

# Lưu các hình ảnh đại diện cho mỗi cụm
for cluster, index in enumerate(centroid_indices):
    print(f"Saving centroid image for cluster {cluster}", end="\r")
    image_name = filelist[index]
    shutil.copy(
        os.path.join(images_root, image_name),
        os.path.join(integration_save_dir, image_name),
    )
    
    # Xuất tệp nhãn tương ứng
    label_name = os.path.splitext(image_name)[0] + '.txt'
    shutil.copy(
        os.path.join(labels_root, label_name),
        os.path.join(integration_save_dir, label_name),
    )

print("\nDone saving centroid images and labels.")

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.
Extracting features: 100%|██████████| 835/835 [00:15<00:00, 55.60it/s]
/home/quangsang/.conda/envs/pytorch/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Saving centroid image for cluster 199
Done saving centroid images and labels.


In [4]:
import os
import shutil

# Đường dẫn tới thư mục chứa các hình ảnh và nhãn đầu vào
images_dir = '/home/quangsang/test/cam1/represent_images/images'
labels_dir = '/home/quangsang/test/cam1/represent_images/labels'
output_dir = '/home/quangsang/test/cam1/represent_images_by_class'

# Tạo thư mục đầu ra nếu chưa tồn tại
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Duyệt qua tất cả các tệp hình ảnh trong thư mục images_dir
for image_name in os.listdir(images_dir):
    if image_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        # Lấy class_id từ tên tệp hình ảnh
        parts = image_name.split('_')
        class_id = parts[-1].split('.')[0].replace('class_', '')

        # Tạo thư mục cho class_id nếu chưa tồn tại
        class_dir = os.path.join(output_dir, f'class_{class_id}')
        if not os.path.exists(class_dir):
            os.makedirs(class_dir)

        # Đường dẫn tới tệp hình ảnh và nhãn đầu ra
        image_output_path = os.path.join(class_dir, image_name)
        label_name = os.path.splitext(image_name)[0] + '.txt'
        label_output_path = os.path.join(class_dir, label_name)

        # Sao chép tệp hình ảnh và nhãn vào thư mục class_id
        shutil.copy(os.path.join(images_dir, image_name), image_output_path)
        shutil.copy(os.path.join(labels_dir, label_name), label_output_path)

print("Done organizing images and labels by class.")

Done organizing images and labels by class.
